In [ ]:
%cd seir

In [ ]:
import pickle

In [ ]:
import pandas as pd
import pymc3 as pm
from tqdm import tqdm

In [ ]:
import holoviews as hv
hv.notebook_extension('bokeh')

In [ ]:
%load_ext autoreload
%autoreload 1
%aimport data, fiddle, seir, util

In [ ]:
import fiddle
import seir
import util

In [ ]:
fiddle.fiddle()

In [ ]:
%%time
recommended, best = seir.fit_nevergrad_model(24_000)
with open('fit.pkl', 'wb') as f:
    pickle.dump((recommended, best), f)

In [ ]:
with open('fit.pkl', 'rb') as f:
    recomended, best = pickle.load(f)
best

In [ ]:
kwargs = best.copy()
kwargs['t0'] = seir.DATE_OF_SIM_TIME_ZERO + pd.to_timedelta(kwargs['t0'], unit='d')
fiddle.draw(**kwargs)

In [ ]:
%%time
model = seir.create_model_from_guess(best)
trace = pm.sample(model=model, draws=24, tune=12, chains=4, cores=4, discard_tuned_samples=False,
                  target_accept=0.9, step_scale=0.1, compute_convergence_checks=False, start=best)
with open('trace.pkl', 'wb') as f:
    pickle.dump(trace, f)
util.plot_trace(trace)